In [1]:
import sys
sys.path.append("../../src")
import os
import datetime
import pandas as pd
import numpy as np
from example_lorenz import get_lorenz_data
from sindy_utils import library_size
from training import train_network
import tensorflow as tf

2026-02-24 06:16:33.484825: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2026-02-24 06:16:33.534182: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2026-02-24 06:16:34.845145: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.


Instructions for updating:
non-resource variables are not supported in the long term


# Generate data

In [2]:
# generate training, validation, testing data
noise_strength = 1e-6
training_data = get_lorenz_data(1024, noise_strength=noise_strength)
validation_data = get_lorenz_data(20, noise_strength=noise_strength)

In [3]:
training_data.keys()

dict_keys(['t', 'y_spatial', 'modes', 'x', 'dx', 'ddx', 'z', 'dz', 'ddz', 'sindy_coefficients'])

In [4]:
np.shape(training_data['sindy_coefficients'])

(20, 3)

# Set up model and training parameters

In [5]:
params = {}

params['input_dim'] = 128      # Input dimension
params['latent_dim'] = 3       # Latent dimension
params['model_order'] = 1      # Order of derivative
params['poly_order'] = 3       # Polynomial order
params['include_sine'] = False # Using sinc functions
params['library_dim'] = library_size(params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_z'] = 0.0
params['loss_weight_sindy_x'] = 1e-4
params['loss_weight_sindy_regularization'] = 1e-5

params['activation'] = 'sigmoid'
# params['widths'] = [64,32]
params['widths'] = [32,16]


# training parameters
params['epoch_size'] = training_data['x'].shape[0]
params['batch_size'] = 1024
params['learning_rate'] = 1e-3

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 5001
params['refinement_epochs'] = 1001

# Run training experiments

In [6]:
num_experiments = 1
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'lorenz_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    # tf.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)
    df = df.append({**results_dict, **params}, ignore_index=True)

df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0
TRAINING


2026-02-24 06:16:52.624952: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
I0000 00:00:1771913812.646090  104413 mlir_graph_optimization_pass.cc:437] MLIR V1 optimization pass is not enabled


Epoch 0
   training loss 0.07223659753799438, (np.float32(0.07058302), np.float32(2809.39), np.float32(16.462698), np.float32(0.7300578))
   validation loss 0.07510649412870407, (np.float32(0.07303791), np.float32(3203.9653), np.float32(20.612865), np.float32(0.7300578))
decoder loss ratio: 0.360532, decoder SINDy loss  ratio: 1.119043
Epoch 100
   training loss 9.640268399380147e-05, (np.float32(4.8818652e-05), np.float32(0.63810915), np.float32(0.28608808), np.float32(1.8975228))
   validation loss 0.0001942967646755278, (np.float32(0.000114826296), np.float32(0.9696319), np.float32(0.6049524), np.float32(1.8975228))
decoder loss ratio: 0.000567, decoder SINDy loss  ratio: 0.032842
Epoch 200
   training loss 4.124381666770205e-05, (np.float32(1.1329353e-05), np.float32(0.0714523), np.float32(0.071274176), np.float32(2.278705))
   validation loss 6.966569344513118e-05, (np.float32(2.7120328e-05), np.float32(0.29391256), np.float32(0.19758318), np.float32(2.278705))
decoder loss ratio:

KeyboardInterrupt: 